In [ ]:
!pip install llm2vec
# !pip install -U "transformers" "huggingface_hub" "tokenizers"
!pip install Levenshtein
!pip install "protobuf==3.20.3" --force-reinstall


In [ ]:
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import os, math, pandas as pd, numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch
import Levenshtein
from typing import List, Sequence, Tuple
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import spearmanr, pearsonr
import editdistance
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from collections import deque
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.cuda.amp import autocast, GradScaler
import kagglehub
import random
from scipy.stats import spearmanr

from types import SimpleNamespace

In [ ]:
TASK_TYPE = "pair_cls"
MAX_LENGTH = 256
BATCH_SIZE = 8
epochs = 5
LR = 2e-5
warmup_ratio = 0.06
save_dir = 'ckpt1'
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

teacher_special="<|embed|>"
student_special="[CLS]"
###



In [ ]:
class TextPairRaw(Dataset):
    def __init__(self, df: pd.DataFrame, task: str):
        self.task = task
        if task == "single_cls":
            self.samples = [(t, None, int(y)) for t, y in zip(df["text"].astype(str), df["label"].astype(int))]
        elif task == "pair_cls":
            self.samples = [(a, b) for a,b in zip(df["premise"].astype(str),
                                                            df["hypothesis"].astype(str))]
        else:  # pair_reg
            self.samples = [(a, b) for a,b in zip(df["sentence1"].astype(str),
                                                              df["sentence2"].astype(str))]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx] 
from typing import List, Tuple, Optional



class DualTokenizerCollate:
    def __init__(self, tok_student, tok_teacher, task: str, max_len: int):
        self.ts = tok_student
        self.tt = tok_teacher
        self.task = task
        self.max_len = max_len

    def __call__(self, batch: List[Tuple[str, Optional[str], float]]):
        s1s, s2s = zip(*batch)

        if self.task == "single_cls":
            s_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)
            t_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)

            out = {
                "input_ids_stu": s_enc["input_ids"],
                "attention_mask_stu": s_enc["attention_mask"],
                "special_tokens_mask_stu": s_enc["special_tokens_mask"],
                "input_ids_tea": t_enc["input_ids"],
                "attention_mask_tea": t_enc["attention_mask"],
                "special_tokens_mask_tea": t_enc["special_tokens_mask"],
                "labels": torch.tensor(ys, dtype=torch.long),
            }
            if "token_type_ids" in s_enc:
                out["token_type_ids_stu"] = s_enc["token_type_ids"]
            if "token_type_ids" in t_enc:
                out["token_type_ids_tea"] = t_enc["token_type_ids"]
            return out

        # ------- pair (bi-encoder) -------
        s1_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        s2_enc = self.ts(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        t1_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        t2_enc = self.tt(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        out = {
            # student
            "input_ids1_stu": s1_enc["input_ids"],
            "attention_mask1_stu": s1_enc["attention_mask"],
            "special_tokens_mask1_stu": s1_enc["special_tokens_mask"],
            "input_ids2_stu": s2_enc["input_ids"],
            "attention_mask2_stu": s2_enc["attention_mask"],
            "special_tokens_mask2_stu": s2_enc["special_tokens_mask"],
            # teacher
            "input_ids1_tea": t1_enc["input_ids"],
            "attention_mask1_tea": t1_enc["attention_mask"],
            "special_tokens_mask1_tea": t1_enc["special_tokens_mask"],
            "input_ids2_tea": t2_enc["input_ids"],
            "attention_mask2_tea": t2_enc["attention_mask"],
            "special_tokens_mask2_tea": t2_enc["special_tokens_mask"],
        }
        # chỉ thêm token_type_ids nếu tồn tại
        if "token_type_ids" in s1_enc:
            out["token_type_ids1_stu"] = s1_enc["token_type_ids"]
        if "token_type_ids" in s2_enc:
            out["token_type_ids2_stu"] = s2_enc["token_type_ids"]
        if "token_type_ids" in t1_enc:
            out["token_type_ids1_tea"] = t1_enc["token_type_ids"]
        if "token_type_ids" in t2_enc:
            out["token_type_ids2_tea"] = t2_enc["token_type_ids"]

        return out



# def smart_read(base, name_csv, name_tsv):
#     p_csv = os.path.join(base, name_csv)
#     p_tsv = os.path.join(base, name_tsv)
#     if os.path.exists(p_csv):
#         return pd.read_csv(p_csv)
#     if os.path.exists(p_tsv):
#         return pd.read_csv(p_tsv, sep="\t")
#     raise FileNotFoundError(f"Không thấy {p_csv} hoặc {p_tsv}")


# TASK_TYPE = "pair_cls"
# train_df = smart_read(BASE_INPUT, "train.csv", "train.tsv")
# dev_df   = smart_read(BASE_INPUT, "dev.csv",   "dev.tsv")
# test_df  = smart_read(BASE_INPUT, "test.csv",  "test.tsv")
# assert {"premise","hypothesis","label"}.issubset(train_df.columns), "SciTail cần premise,hypothesis"
print("Done Prepare Dataset")

In [ ]:
def info_nce(q, k, temperature=0.07, neg_valid_mask=None):
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)

    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss_inbatch = F.cross_entropy(logits, labels) 
    return loss_inbatch, logits

In [ ]:
data_train = '/kaggle/input/multitask-data/merged_9_data_3k_each_ver2.csv'
df = pd.read_csv(data_train)
df.head(3)

In [ ]:
df["premise"] = df["text"]
df["hypothesis"] = df["text"]
cols = ["premise", "hypothesis"]  # hoặc ["premise","hypothesis","label"] nếu cần placeholder
df_out = df[cols].copy()

In [ ]:
student_name = "huawei-noah/TinyBERT_General_4L_312D"
tok_student = AutoTokenizer.from_pretrained(student_name)
model_student = AutoModel.from_pretrained(
    student_name,
    output_hidden_states=True
)

tok_teacher = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_teacher = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)

In [ ]:

train_ds = TextPairRaw(df_out, TASK_TYPE)

collate = DualTokenizerCollate(tok_student, tok_teacher, TASK_TYPE, MAX_LENGTH)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate, pin_memory=True, num_workers=2, persistent_workers=True)
print("Done Prepare Dataloader")

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from scipy.stats import pearsonr, spearmanr

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class STSDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['score'], dtype=torch.float),
        }
        
def collate_fn(batch, tokenizer, max_len=128):
    s1_list = [item["sentence1"] for item in batch]
    s2_list = [item["sentence2"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )
    enc2 = tokenizer(
        s2_list,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "input_ids2": enc2["input_ids"],
        "attention_mask2": enc2["attention_mask"],
        "labels": labels,
    }

def eval_sts(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                # emb1 = mean_pooling(out1, attn1)
                # emb2 = mean_pooling(out2, attn2)
                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) * 2.5  # scale [-1,1] -> [0,5]
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    spearman_corr, _ = spearmanr(preds, labels)
    print(f"Spearman: {spearman_corr:.4f}")

    return spearman_corr


def eval_sts_task(model, path_list):
    model.eval()
    print(' ✅ eval_sts_task')
    for path in path_list:
        print(path)
        eval_dataset = STSDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_sts(model, eval_loader)
    model.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import datasets
import numpy as np
import torch

def eval_cls(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                label = batch["labels"]

                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                emb1 = out1.last_hidden_state[:, 0, :]
        
                preds.extend(emb1.cpu().numpy())
                labels.extend(label.numpy())
    
    return preds, labels

class ClasssifyDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return {
            "text": self.dataset.iloc[idx]['text'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.long),
        }

def clf_collate_fn(batch, tokenizer, max_len=512):
    s1_list = [item["text"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "labels": labels,
    }


def eval_classification_task(model, path_list):
    model.eval()
    print(' ✅ eval classifier')

    for train_path, dev_path in path_list:
        print(dev_path)
        eval_dataset = ClasssifyDataset(dev_path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )
        
        train_dataset = ClasssifyDataset(train_path)
        train_loader = DataLoader(
            train_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )

        X_train, y_train = eval_cls(model, train_loader)
        X_test, y_test = eval_cls(model, eval_loader)

        clf = LogisticRegression(
            random_state=42,
            n_jobs=1,
            max_iter=200,
            verbose=0,
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        scores = {}
        accuracy = accuracy_score(y_test, y_pred)
        scores["accuracy"] = accuracy
        f1 = f1_score(y_test, y_pred, average="macro")
        scores["f1"] = f1
        print(scores)
        
    model.train()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score


tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class PairDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.float),
        }
        

def eval_pair(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) / 2
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    metric = get_metric_pair_classification(preds, labels)
    print(metric)

    return metric

def get_metric_pair_classification(scores, labels):
    best_acc, best_thr = 0, 0
    for thr in np.linspace(0, 1, 200):
        preds = (scores >= thr).astype(int)
        acc = accuracy_score(labels, preds)
        if acc > best_acc:
            best_acc, best_thr = acc, thr
    preds = (scores >= best_thr).astype(int)
    return {
        "best_threshold": best_thr,
        "accuracy": best_acc,
        "f1": f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "average_precision": average_precision_score(labels, scores)
    }


def eval_pair_task(model, path_list):
    model.eval()
    print(' ✅ eval_pair_task')
    for path in path_list:
        print(path)
        eval_dataset = PairDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_pair(model, eval_loader)
    model.train()

In [ ]:
# import editdistance
# import re
# import math

# def preprocess_text(text):

#     # Remove numbers if specified
#     text = re.sub(r'\d+', '', text)

#     # Custom list of English stopwords (a common subset)
#     stop_words = [
#         'a', 'an', 'the', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through',
#         'during', 'here', 'there', 'all', 'any', 'both', 'each', 'few', 'other', 'such',
#         'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', 'now',
#         'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
#         'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
#         'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
#         'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
#         'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
#         'did', 'doing', 'would', 'could', 'should', 'ought', 'i\'m', 'you\'re', 'he\'s',
#         'she\'s', 'it\'s', 'we\'re', 'they\'re', 'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve',
#         'i\'d', 'you\'d', 'he\'d', 'she\'d', 'we\'d', 'they\'d', 'i\'ll', 'you\'ll', 'he\'ll',
#         'she\'ll', 'we\'ll', 'they\'ll', 'let\'s', 'that\'s', 'who\'s', 'what\'s', 'here\'s', 'there\'s', 'when\'s', 'where\'s',
#         'why\'s', 'how\'s', '.'
        
#     ]


#     words = [word for word in text.split() if word not in stop_words]
#     text = ' '.join(words)

#     return text
# def find_best_mapping(x, base_tokens, blending_special, base_special, best_one=True):
#     tmp_x = x.replace(blending_special, base_special)
#     if tmp_x in base_tokens:
#         return tmp_x, tmp_x
#     else:
#         if best_one:
#             best = None
#             best_dist = None
#             for y in base_tokens:
#                 d = editdistance.eval(tmp_x, y)
#                 if best is None or d < best_dist:
#                     best = y
#                     best_dist = d
#             return tmp_x, best
#         else:
#             token_and_distance = [(y, editdistance.eval(tmp_x, y)) for y in base_tokens]
#             min_distance = min(d for _, d in token_and_distance)
#             shortest_distance_tokens = [y for y, d in token_and_distance if d == min_distance]
#             return tmp_x, shortest_distance_tokens
# class CKALoss(nn.Module):
#     """
#     Loss with knowledge distillation.
#     """
#     def __init__(self, eps ):
#         super().__init__()
#         self.eps = eps
#     def forward(self, SH, TH): 
#         dT = TH.size(-1)
#         dS = SH.size(-1)
#         SH = SH.view(-1,dS).to(SH.device,torch.float64)
#         TH = TH.view(-1,dT).to(SH.device,torch.float64)
        
#         slen = SH.size(0)
#                 # Dropout on Hidden State Matching
#         SH = SH - SH.mean(0, keepdim=True)
#         TH = TH - TH.mean(0, keepdim=True)
                
#         num = torch.norm(SH.t().matmul(TH),'fro')
#         den1 = torch.norm(SH.t().matmul(SH),'fro') + self.eps
#         den2 = torch.norm(TH.t().matmul(TH),'fro') + self.eps
        
#         return 1 - num/torch.sqrt(den1*den2)

# # Hàm ánh xạ token song hướng giữa teacher và student
# def align_text_tokens(text):
#     # Giả sử tok_teacher và tok_student đã được khởi tạo
#     teacher_tokens = set(tok_teacher.tokenize(text))
#     student_tokens = set(tok_student.tokenize(text))
#     teacher_special = TOKENIZER_TO_SPECIAL_TOKEN[type(tok_teacher)]
#     student_special = TOKENIZER_TO_SPECIAL_TOKEN[type(tok_student)]

#     teacher_to_student = {}
#     for t in teacher_tokens:
#         _, s = find_best_mapping(t, student_tokens, teacher_special, student_special, best_one=True)
#         teacher_to_student[t] = s

#     student_to_teacher = {}
#     for s in student_tokens:
#         _, t = find_best_mapping(s, teacher_tokens, student_special, teacher_special, best_one=True)
#         student_to_teacher[s] = t

#     reciprocal_mapping = {}
#     for t, s in teacher_to_student.items():
#         if s in student_to_teacher and student_to_teacher[s] == t:
#             reciprocal_mapping[t] = s

#     return reciprocal_mapping
# def extract_top_k_tokens_from_att(last_layer_attention, input_ids_teacher, tokenizer_teacher, k):
#     avg_attention = last_layer_attention.mean(dim=0)  # [L, L]
#     token_importance = avg_attention.sum(dim=0).to(torch.float32)
#     tokens = tokenizer_teacher.convert_ids_to_tokens(input_ids_teacher.detach().cpu().tolist())
#     token_importance_pairs = list(zip(tokens, token_importance.tolist()))

#     # Sắp xếp giảm dần theo importance và lấy top k
#     k = max(1, min(k, len(token_importance_pairs)))  # tránh k=0 hoặc k>len
#     top_k_tokens = sorted(token_importance_pairs, key=lambda x: x[1], reverse=True)[:k]

#     # Trả về SET tên token để tiện lọc
#     top_k_tokens_set = {token for token, _ in top_k_tokens}
#     return top_k_tokens_set



# def get_top_k_reciprocal_mapping(text, last_layer_attention, input_ids_teacher, tokenizer_teacher):
#     # Lấy ánh xạ song phương giữa teacher và student
#     reciprocal_mapping = align_text_tokens(text)
#     n = len(reciprocal_mapping)
    
#     k = max(1, n // 3)
#     top_k_tokens_set = extract_top_k_tokens_from_att(
#         last_layer_attention,      # [H, L_t, L_t]
#         input_ids_teacher,         # [L_t]
#         tokenizer_teacher,         # tok_teacher
#         k
#     )
#     # Lọc reciprocal mapping chỉ giữ các token teacher có trong top k
#     reciprocal_mapping_top_k = {t: s for t, s in reciprocal_mapping.items() if t in top_k_tokens_set}
#     return reciprocal_mapping_top_k



# # Hàm lấy chỉ số (indices) từ ánh xạ reciprocal_mapping
# def get_indices_from_mapping(text, reciprocal_mapping):
#     input_ids_teacher = tok_teacher.encode(text, return_tensors='pt')[0]
#     input_ids_student = tok_student.encode(text, return_tensors='pt')[0]
    
#     # Tạo tập hợp các token_id duy nhất từ reciprocal_mapping
#     teacher_token_ids = {tok_teacher.convert_tokens_to_ids(t) for t in reciprocal_mapping.keys()}
#     student_token_ids = {tok_student.convert_tokens_to_ids(s) for s in reciprocal_mapping.values()}
    
#     # Chọn chỉ số đầu tiên cho mỗi token_id trong teacher
#     teacher_indices = []
#     seen_teacher = set()  # Theo dõi các token_id đã xử lý
#     for idx, token_id in enumerate(input_ids_teacher):
#         tid = token_id.item()
#         if tid in teacher_token_ids and tid not in seen_teacher:
#             teacher_indices.append(idx)
#             seen_teacher.add(tid)
#     # Chọn chỉ số đầu tiên cho mỗi token_id trong student
#     student_indices = []
#     seen_student = set()  # Theo dõi các token_id đã xử lý
#     for idx, token_id in enumerate(input_ids_student):
#         tid = token_id.item()
#         if tid in student_token_ids and tid not in seen_student:
#             student_indices.append(idx)
#             seen_student.add(tid)
    
#     return teacher_indices, student_indices


# def compute_att_loss_2(teacher_atts, student_atts,
#                        input_ids_tea, input_ids_stu,
#                        k, tok_teacher, tok_student, device):
#     att_loss_total = 0.0
#     batch_size = input_ids_stu.size(0)

#     def decode_input_ids(tokenizer, input_ids):
#         return tokenizer.decode(input_ids, skip_special_tokens=True)

#     teacher_layer_num = len(teacher_atts)
#     student_layer_num = len(student_atts)
#     layers_per_block = teacher_layer_num // student_layer_num
#     new_teacher_atts = [teacher_atts[idx * layers_per_block + layers_per_block - 1]
#                         for idx in range(student_layer_num)]

#     teacher_last_k_layers = new_teacher_atts[-k:]   # list length k
#     student_last_k_layers = student_atts[-k:]       # list length k

#     for i in range(batch_size):
#         # 1) Text từ student input để align token
#         text = decode_input_ids(tok_student, input_ids_stu[i].cpu())
#         text = text.lower()
#         text = re.sub(r'[^\w\s]', '', text)

#         # 2) Lấy attention layer cuối của teacher cho sample i
#         #    teacher_atts[-1]: [B, H, L_t, L_t]
#         last_layer_att_i = teacher_atts[-1][i]   # [H, L_t, L_t]

#         # 3) Gọi get_top_k_reciprocal_mapping với att + input_ids_tea[i]
#         reciprocal_mapping_top_k = get_top_k_reciprocal_mapping(
#             text,
#             last_layer_att_i,
#             input_ids_tea[i],
#             tok_teacher
#         )

#         if len(reciprocal_mapping_top_k) == 0:
#             continue  # không map được thì skip sample này

#         teacher_indices, student_indices = get_indices_from_mapping(text, reciprocal_mapping_top_k)

#         for teacher_att, student_att in zip(teacher_last_k_layers, student_last_k_layers):
#             # teacher_att: [B, H, L_t, L_t], student_att: [B, H, L_s, L_s]
#             teacher_att_for_k_token = teacher_att[i, :, teacher_indices, :].mean(dim=0)  # [len(teacher_indices), L_t] -> [k, L_t]
#             student_att_for_k_token = student_att[i, :, student_indices, :].mean(dim=0)  # [len(student_indices), L_s]

#             teacher_att_for_k_token = torch.where(
#                 teacher_att_for_k_token <= -1e2,
#                 torch.zeros_like(teacher_att_for_k_token, device=device),
#                 teacher_att_for_k_token
#             )
#             student_att_for_k_token = torch.where(
#                 student_att_for_k_token <= -1e2,
#                 torch.zeros_like(student_att_for_k_token, device=device),
#                 student_att_for_k_token
#             )

#             cka = CKALoss(eps=1e-8).to(device)
#             att_loss_total += cka(student_att_for_k_token, teacher_att_for_k_token)

#     return att_loss_total



# # def compute_att_loss_2(teacher_atts, student_atts, input_ids_tea, input_ids_stu, k, tok_teacher, tok_student, device):
# #     att_loss_total = 0.0

# #     # Lấy batch_size từ input_ids
# #     batch_size = input_ids_stu.shape[0]

# #     # Hàm decode input_ids thành văn bản
# #     def decode_input_ids(tokenizer, input_ids):
# #         return tokenizer.decode(input_ids, skip_special_tokens=True)

# #     # Duyệt qua từng sample trong batch
# #     for i in range(batch_size):
# #         text = decode_input_ids(tok_student, input_ids_stu[i].cpu())
# #         text = text.lower()

# #         text = re.sub(r'[^\w\s]', '', text)
        
# #         reciprocal_mapping_top_k = get_top_k_reciprocal_mapping(text)
# #         teacher_indices, student_indices = get_indices_from_mapping(text, reciprocal_mapping_top_k)

#     #     teacher_outputs = teacher_model(input_ids_teacher, attention_mask=attention_mask_teacher, output_attentions=True)
#     #     student_outputs = student_model(input_ids_student, attention_mask=attention_mask_student, output_attentions=True)
        
#     #     teacher_layer_num = len(teacher_atts)
#     #     student_layer_num = len(student_atts)
#     #     layers_per_block = teacher_layer_num // student_layer_num

#     #     new_teacher_atts = [teacher_atts[i * layers_per_block + layers_per_block - 1] for i in range(student_layer_num)]

#     #     # Lấy k layer cuối (k tương ứng với số layer sử dụng để tính loss)
#     #     teacher_last_k_layers = new_teacher_atts[-k:]
#     #     student_last_k_layers = student_atts[-k:]
#     #     # print('Bắt đầu vào vòng lặp tính loss')

#     #     # Lặp qua từng layer trong k layer cuối
#     #     for teacher_att, student_att in zip(teacher_last_k_layers, student_last_k_layers):

#     #         teacher_att_for_k_token = teacher_att[0, :, teacher_indices, :].mean(dim=0)  # (k, t)
#     #         student_att_for_k_token = student_att[0, :, student_indices, :].mean(dim=0)   # (k, s)

#     #         # Xử lý các giá trị attention nhỏ
#     #         teacher_att_for_k_token = torch.where(
#     #             teacher_att_for_k_token <= -1e2,
#     #             torch.zeros_like(teacher_att_for_k_token).to(device),
#     #             teacher_att_for_k_token
#     #         )
#     #         student_att_for_k_token = torch.where(
#     #             student_att_for_k_token <= -1e2,
#     #             torch.zeros_like(student_att_for_k_token).to(device),
#     #             student_att_for_k_token
#     #         )
            
#     #         cka_loss_fn = CKALoss(eps=1e-8).to(device)
#     #         cka_loss = cka_loss_fn(student_att_for_k_token, teacher_att_for_k_token)
#     #         att_loss_total  += cka_loss   

#     # return att_loss_total

In [ ]:
import torch
import torch.nn as nn
import math
def find_best_mapping(x, base_tokens, blending_special, base_special, best_one=True):
    tmp_x = x.replace(blending_special, base_special)
    if tmp_x in base_tokens:
        return tmp_x, tmp_x
    else:
        if best_one:
            best = None
            best_dist = None
            for y in base_tokens:
                d = editdistance.eval(tmp_x, y)
                if best is None or d < best_dist:
                    best = y
                    best_dist = d
            return tmp_x, best
        else:
            token_and_distance = [(y, editdistance.eval(tmp_x, y)) for y in base_tokens]
            min_distance = min(d for _, d in token_and_distance)
            shortest_distance_tokens = [y for y, d in token_and_distance if d == min_distance]
            return tmp_x, shortest_distance_tokens
class CKALoss(nn.Module):
    def __init__(self, eps ):
        super().__init__()
        self.eps = eps
    def forward(self, SH, TH): 
        dT = TH.size(-1)
        dS = SH.size(-1)
        SH = SH.view(-1,dS).to(SH.device,torch.float64)
        TH = TH.view(-1,dT).to(SH.device,torch.float64)
        
        slen = SH.size(0)
        SH = SH - SH.mean(0, keepdim=True)
        TH = TH - TH.mean(0, keepdim=True)
                
        num = torch.norm(SH.t().matmul(TH),'fro')
        den1 = torch.norm(SH.t().matmul(SH),'fro') + self.eps
        den2 = torch.norm(TH.t().matmul(TH),'fro') + self.eps
        
        return 1 - num/torch.sqrt(den1*den2)
def build_reciprocal_mapping_from_token_lists(
    teacher_tokens, student_tokens,
    teacher_special="<s>", student_special="[CLS]"
):
    teacher_to_student = align_tokens(teacher_tokens, student_tokens, teacher_special, student_special)
    student_to_teacher = align_tokens(student_tokens, teacher_tokens, student_special, teacher_special)

    reciprocal_mapping = {}
    for t, s in teacher_to_student.items():
        if s in student_to_teacher and student_to_teacher[s] == t:
            reciprocal_mapping[t] = s
    return reciprocal_mapping

def filter_mapping_top_k(teacher_tokens, importance_vec, reciprocal_mapping, k):
    if len(reciprocal_mapping) == 0:
        return {}

    token_scores = {}
    for idx, tok in enumerate(teacher_tokens):
        if tok in reciprocal_mapping:
            score = importance_vec[idx].item()
            if tok not in token_scores or score > token_scores[tok]:
                token_scores[tok] = score

    if len(token_scores) == 0:
        return {}

    k = max(1, min(k, len(token_scores)))
    top_tokens = sorted(token_scores.items(), key=lambda x: x[1], reverse=True)[:k]
    allowed = {tok for tok, _ in top_tokens}

    filtered = {t: s for t, s in reciprocal_mapping.items() if t in allowed}
    return filtered

def get_indices_from_mapping_tokens(teacher_tokens, student_tokens, reciprocal_mapping):
    if len(reciprocal_mapping) == 0:
        return [], []

    teacher_indices = [idx for idx, tok in enumerate(teacher_tokens) if tok in reciprocal_mapping]

    mapped_student_tokens = set(reciprocal_mapping.values())
    student_indices = [idx for idx, tok in enumerate(student_tokens) if tok in mapped_student_tokens]

    return teacher_indices, student_indices

def align_indices_by_mapping(teacher_indices, student_indices, reciprocal_top, teacher_tokens, student_tokens):
    aligned_teacher = []
    aligned_student = []

    s2t = {v:k for k,v in reciprocal_top.items()}

    for t_idx in teacher_indices:
        t_tok = teacher_tokens[t_idx]
        if t_tok not in reciprocal_top:
            continue
        s_tok = reciprocal_top[t_tok]
        for idx_s, tok_s in enumerate(student_tokens):
            if tok_s == s_tok:
                aligned_teacher.append(t_idx)
                aligned_student.append(idx_s)
                break

    return aligned_teacher, aligned_student

def compute_att_loss_2(
    teacher_atts,          # list: mỗi phần tử [B, H, L_t, L_t] trên device_s
    student_atts,          # list: mỗi phần tử [B, H, L_s, L_s]
    input_ids_tea,         # [B, L_t] trên device_s
    input_ids_stu,         # [B, L_s] trên device_s
    attention_mask_tea,    # [B, L_t] trên device_s
    attention_mask_stu,    # [B, L_s] trên device_s
    tok_teacher,
    tok_student,
    k,                     # số last layers dùng (thường = 1)
    device,
    teacher_special="<s>",
    student_special="[CLS]",
):

    att_loss_total = 0.0
    batch_size = input_ids_stu.size(0)

    teacher_layer_num = len(teacher_atts)
    student_layer_num = len(student_atts)
    layers_per_block = teacher_layer_num // student_layer_num
    new_teacher_atts = [
        teacher_atts[idx * layers_per_block + layers_per_block - 1]
        for idx in range(student_layer_num)
    ]

    teacher_last_k_layers = new_teacher_atts[-k:]   
    student_last_k_layers = student_atts[-k:]       

    cka = CKALoss(eps=1e-8).to(device)

    for b in range(batch_size):
        L_t_valid = int(attention_mask_tea[b].sum().item())
        L_s_valid = int(attention_mask_stu[b].sum().item())
        if L_t_valid == 0 or L_s_valid == 0:
            continue

        ids_tea = input_ids_tea[b, :L_t_valid]
        ids_stu = input_ids_stu[b, :L_s_valid]

        teacher_tokens = tok_teacher.convert_ids_to_tokens(ids_tea.detach().cpu().tolist())
        student_tokens = tok_student.convert_ids_to_tokens(ids_stu.detach().cpu().tolist())

        last_teacher_att_full = teacher_atts[-1][b]        # [H, L_t, L_t]
        last_teacher_att = last_teacher_att_full[:, :L_t_valid, :L_t_valid]

        teacher_importance = compute_token_importance(
            last_teacher_att, teacher_tokens
        )  # [L_t_valid]

        reciprocal = build_reciprocal_mapping_from_token_lists(
            teacher_tokens, student_tokens,
            teacher_special=teacher_special,
            student_special=student_special
        )
        if len(reciprocal) == 0:
            continue

        n_map = len(reciprocal)
        k_top = max(1, n_map // 3)

        token_scores = {}
        for idx_t, tok_t in enumerate(teacher_tokens):
            if tok_t in reciprocal:
                score = teacher_importance[idx_t].item()
                if tok_t not in token_scores or score > token_scores[tok_t]:
                    token_scores[tok_t] = score

        if len(token_scores) == 0:
            continue

        top_tokens = sorted(token_scores.items(), key=lambda x: x[1], reverse=True)[:k_top]
        allowed_teacher_tokens = {t for t, _ in top_tokens}

        aligned_teacher_indices = []
        aligned_student_indices = []

        for t_tok in allowed_teacher_tokens:
            s_tok = reciprocal[t_tok]
            # teacher index: lấy xuất hiện đầu tiên
            try:
                t_idx = teacher_tokens.index(t_tok)
            except ValueError:
                continue
            # student index: xuất hiện đầu tiên
            try:
                s_idx = student_tokens.index(s_tok)
            except ValueError:
                continue
            if t_idx < L_t_valid and s_idx < L_s_valid:
                aligned_teacher_indices.append(t_idx)
                aligned_student_indices.append(s_idx)

        N = len(aligned_teacher_indices)
        if N == 0:
            continue

        for teacher_att_layer, student_att_layer in zip(teacher_last_k_layers, student_last_k_layers):
            # teacher_att_layer: [B, H, L_t, L_t]
            # student_att_layer: [B, H, L_s, L_s]

            tea_sub = teacher_att_layer[b, :, :L_t_valid, :L_t_valid]  # [H, L_t_valid, L_t_valid]
            stu_sub = student_att_layer[b, :, :L_s_valid, :L_s_valid]  # [H, L_s_valid, L_s_valid]

            # attention của N token đã align, trung bình các head
            tea_tok_att = tea_sub[:, aligned_teacher_indices, :].mean(dim=0)  # [N, L_t_valid]
            stu_tok_att = stu_sub[:, aligned_student_indices, :].mean(dim=0)  # [N, L_s_valid]

            # xử lý mask rất nhỏ
            tea_tok_att = torch.where(
                tea_tok_att <= -1e2,
                torch.zeros_like(tea_tok_att, device=device),
                tea_tok_att
            )
            stu_tok_att = torch.where(
                stu_tok_att <= -1e2,
                torch.zeros_like(stu_tok_att, device=device),
                stu_tok_att
            )

            # 10) ép feature dim giống nhau để CKA không lỗi
            d_s = stu_tok_att.size(1)
            d_t = tea_tok_att.size(1)
            d = min(d_s, d_t)
            if d == 0:
                continue

            SH = stu_tok_att[:, :d]  # [N, d]
            TH = tea_tok_att[:, :d]  # [N, d]

            att_loss_total += cka(SH, TH)

    return att_loss_total



In [ ]:
import torch

def sinkhorn(
    cost_matrix,          # [m, n]
    a,                    # [m, 1] hoặc [m]
    b,                    # [n, 1] hoặc [n]
    alpha=0.1,            # self.sinkhorn_alpha
    max_iter=100,         # self.OT_max_iter
    stop_thr=1e-9,        # self.stopThr
    eps=1e-9              # self.epsilon
):
    """
    Sinkhorn Optimal Transport (không cần class).
    Mọi tensor phải nằm trên cùng device.
    """

    m, n = cost_matrix.shape
    device = cost_matrix.device
    dtype = cost_matrix.dtype

    if m == 0 or n == 0:
        return (
            torch.tensor(0.0, device=device, dtype=dtype),
            torch.zeros((m, n), device=device, dtype=dtype)
        )

    a = a.to(device=device, dtype=dtype)
    b = b.to(device=device, dtype=dtype)

    if a.dim() == 1:
        a = a.view(-1, 1)
    if b.dim() == 1:
        b = b.view(-1, 1)
    if a.shape[0] != m:
        a = torch.ones(m, 1, device=device, dtype=dtype) / m
    if b.shape[0] != n:
        b = torch.ones(n, 1, device=device, dtype=dtype) / n

    if torch.sum(a) < eps or torch.sum(b) < eps:
        a = torch.ones(m, 1, device=device, dtype=dtype) / m
        b = torch.ones(n, 1, device=device, dtype=dtype) / n
    else:
        a = a / torch.sum(a)
        b = b / torch.sum(b)

    K = torch.exp(-cost_matrix / alpha) 
    u = torch.ones(m, 1, device=device, dtype=dtype)
    v = torch.ones(n, 1, device=device, dtype=dtype)
    for _ in range(max_iter):
        u_prev = u.clone()
        KTu = torch.matmul(K.t(), u)  # [n,1]
        v = b / (KTu + eps)
        Kv = torch.matmul(K, v)  # [m,1]
        u = a / (Kv + eps)

        # Check convergence
        err = torch.norm(u - u_prev, p=float('inf'))
        if err < stop_thr:
            break

    # ====== 8. Transport matrix ======
    P = torch.diag(u.squeeze()) @ K @ torch.diag(v.squeeze())  # [m,n]

    # ====== 9. Compute OT loss ======
    ot_loss = torch.sum(P * cost_matrix)

    return ot_loss, P

def pairwise_attention_distance(x, y, eps=1e-8):
    d = x.shape[1]
    sim_mt = torch.mm(x, y.transpose(0, 1)) / math.sqrt(d)
    attention_weights = torch.softmax(sim_mt, dim=1)
    dist_mt = 1.0 - attention_weights
    return dist_mt
def project_importance(teacher_importance, teacher_tokens, student_tokens, mapping):
    device = teacher_importance.device
    student_importance = torch.zeros(len(student_tokens), device=device)
    
    # Get valid teacher tokens based on attention mask
    valid_teacher_tokens = teacher_tokens[:teacher_importance.shape[0]]
    
    # Map valid tokens to importance scores
    teacher_token_to_importance = {token: score.item() for token, score in zip(valid_teacher_tokens, teacher_importance)}
    
    # Keep track of mapped student indices
    mapped_student_indices = set()
    
    # Project importance scores
    for t_idx, t in enumerate(valid_teacher_tokens):
        if t in mapping:
            s = mapping[t]
            # Find all occurrences of this student token
            s_indices = [i for i, token in enumerate(student_tokens) if token == s]
            for s_idx in s_indices:
                if s_idx < len(student_importance):  # Ensure index is valid
                    student_importance[s_idx] = teacher_importance[t_idx]
                    mapped_student_indices.add(s_idx)
    
    # Find minimum importance score from teacher for unmapped tokens
    min_importance = teacher_importance.min().item() if len(teacher_importance) > 0 else 0.0
    for s_idx in range(len(student_tokens)):
        if s_idx not in mapped_student_indices and s_idx < len(student_importance):
            student_importance[s_idx] = min_importance
    student_importance = torch.softmax(student_importance, dim=0)
    
    return student_importance
def align_tokens(teacher_tokens, student_tokens, teacher_special="<s>", student_special="[CLS]"):
    # Create mapping dictionary
    teacher_to_student = {}
    
    # Handle empty token lists
    if not teacher_tokens or not student_tokens:
        return teacher_to_student
    if teacher_special in teacher_tokens and student_special in student_tokens:
        teacher_to_student[teacher_special] = student_special
    student_token_set = set(student_tokens)
    
    for t in teacher_tokens:
        tmp_t = t.replace(teacher_special, student_special)
        if tmp_t in student_token_set:
            teacher_to_student[t] = tmp_t
            continue
        
        best_s = None
        best_dist = float('inf')
        
        for s in student_tokens:
            if s == student_special:
                continue
                
            # Calculate edit distance
            d = editdistance.eval(tmp_t, s)
            if d < best_dist:
                best_s = s
                best_dist = d
        if best_s is not None:
            teacher_to_student[t] = best_s
    
    return teacher_to_student
def compute_token_importance(attention_weights, tokens):
    device = attention_weights.device
    
    # Check if attention_weights is 3D (with multiple heads) or 2D (single attention matrix)
    if len(attention_weights.shape) == 3:
        # Average attention across heads: [seq_len, seq_len]
        avg_attention = attention_weights.mean(dim=0)
    else:
        # Already a 2D attention matrix
        avg_attention = attention_weights
    
    # Ensure dimensions match
    seq_len = min(avg_attention.shape[0], len(tokens))
    
    # Truncate attention matrix if needed
    avg_attention = avg_attention[:seq_len, :seq_len]
    
    # Sum attention that each token receives: [seq_len]
    token_importance = avg_attention.sum(dim=0)
    
    # Normalize importance scores (add small epsilon to avoid division by zero)
    norm_importance = torch.softmax(token_importance, dim=0)
    
    return norm_importance

def compute_ot_loss(
    teacher_last,           # [B, L_t, d_t]  trên device_s
    student_last,           # [B, L_s, d_s]  trên device_s
    teacher_att_last,       # [B, H, L_t, L_t] (attention layer cuối) trên device_s
    attention_mask_teacher, # [B, L_t] trên device_s
    attention_mask_student, # [B, L_s] trên device_s
    input_ids_tea,          # [B, L_t] trên device_s
    input_ids_stu,          # [B, L_s] trên device_s
    tok_teacher,
    tok_student,
    projector,              # proj_t2s: Linear(d_t -> d_s)
    teacher_special="<s>",
    student_special="[CLS]",
):
    device = teacher_last.device
    batch_size = teacher_last.size(0)
    total_loss = 0.0

    for b in range(batch_size):
        valid_teacher_len = int(attention_mask_teacher[b].sum().item())
        valid_student_len = int(attention_mask_student[b].sum().item())

        valid_teacher_input_ids = input_ids_tea[b, :valid_teacher_len]
        valid_student_input_ids = input_ids_stu[b, :valid_student_len]

        teacher_tokens = tok_teacher.convert_ids_to_tokens(
            valid_teacher_input_ids.detach().cpu().tolist()
        )
        student_tokens = tok_student.convert_ids_to_tokens(
            valid_student_input_ids.detach().cpu().tolist()
        )

        teacher_seq = teacher_last[b, :valid_teacher_len, :]   # [Lt', d_t]
        student_seq = student_last[b, :valid_student_len, :]   # [Ls', d_s]

        projected_teacher_seq = projector(teacher_seq)         # [Lt', d_s]

        teacher_attention_full = teacher_att_last[b]           # [H, L_t, L_t]
        valid_teacher_attention = teacher_attention_full[:, :valid_teacher_len, :valid_teacher_len]

        teacher_importance = compute_token_importance(
            valid_teacher_attention, teacher_tokens
        )  # [Lt']

        token_mapping = align_tokens(
            teacher_tokens, student_tokens,
            teacher_special=teacher_special,
            student_special=student_special
        )

        student_importance = project_importance(
            teacher_importance,
            teacher_tokens,
            student_tokens,
            token_mapping
        )   # [Ls']

        tea_mass = teacher_importance.view(-1, 1).to(device=device, dtype=torch.float32)
        stu_mass = student_importance.view(-1, 1).to(device=device, dtype=torch.float32)

        student_seq_f = student_seq.to(device=device, dtype=torch.float32)
        proj_teacher_seq_f = projected_teacher_seq.to(device=device, dtype=torch.float32)

        cost_matrix = pairwise_attention_distance(student_seq_f, proj_teacher_seq_f)
        cost_matrix = cost_matrix.to(device=device, dtype=torch.float32)

        ot_loss_b, _ = sinkhorn(cost_matrix, stu_mass, tea_mass)
        total_loss += ot_loss_b

    avg_loss = total_loss / batch_size
    return avg_loss

# def compute_ot_loss(
#     teacher_last, student_last, attention_mask_student, attention_mask_teacher, input_ids_tea, input_ids_stu,
#     tok_teacher, tok_student, projector,
#     teacher_special, student_special, logits=False
# ):
#     # Get the last hidden state from both models
#     student_features = outputs.hidden_states[-1]  # Shape: (batch_size, seq_len, hidden_dim)
#     teacher_features = teacher_outputs.hidden_states[-1]  # Shape: (batch_size, seq_len, hidden_dim)
    
#     # Extract and determine target dtype (float32 for highest precision)
#     target_dtype = torch.float32  # Use a consistent dtype for all tensors
    
#     batch_size = teacher_features.size(0)
#     total_loss = 0
#     for b in range(batch_size):
#         teacher_input_ids = input_data["input_ids1_tea"][b]
#         student_input_ids = input_data["input_ids1_stu"][b]
        
        # valid_teacher_len = int(attention_mask_teacher[b].sum().item())
        # valid_teacher_input_ids = teacher_input_ids[:valid_teacher_len]
        
        # valid_student_len = int(attention_mask_student[b].sum().item())
        # valid_student_input_ids = student_input_ids[:valid_student_len]
        
        # teacher_tokens = tok_teacher.convert_ids_to_tokens(valid_teacher_input_ids)
        # student_tokens = tok_student.convert_ids_to_tokens(valid_student_input_ids)
        
        # # Get sequences for current batch
        # teacher_seq = teacher_features[b]  # Shape: (seq_len, hidden_dim)
        # student_seq = student_features[b]  # Shape: (seq_len, hidden_dim)

        # # Get masks for current batch
        # teacher_mask = attention_mask_teacher[b]  # (seq_len)
        # student_mask = attention_mask_student[b]  # (seq_len)
        
        # # Prune sequences based on the mask
        # valid_teacher_seq = teacher_seq[teacher_mask.bool()]  # Shape: (valid_seq_len, hidden_dim)
        # valid_student_seq = student_seq[student_mask.bool()]  # Shape: (valid_seq_len, hidden_dim)
        
        # # Project each row of teacher_seq to student space
        # projected_teacher_seq = projector(valid_teacher_seq)  # Now project after pruning
        
        # # Ensure both tensors are in the same dtype
        # dtype = valid_student_seq.dtype
        # projected_teacher_seq = projected_teacher_seq.to(dtype)
        
        # # Process attention weights
        # if hasattr(teacher_outputs, 'attentions') and teacher_outputs.attentions is not None:
        #     teacher_attention = teacher_outputs.attentions[-1][b]
            
        #     # Ensure teacher_attention has the right shape for current batch
        #     valid_teacher_attention = teacher_attention[:, :valid_teacher_len, :valid_teacher_len]
            
    #         # Compute token importance from teacher attention
    #         teacher_importance = compute_token_importance(valid_teacher_attention, teacher_tokens[:valid_teacher_len])
    #     else:
    #         # Fallback if attentions not available
    #         teacher_importance = torch.ones(len(teacher_tokens), device=teacher_seq.device)
    #         teacher_importance = torch.softmax(teacher_importance, dim=0)
        
    #     token_mapping = align_tokens(teacher_tokens, student_tokens, 
    #                                   teacher_special, student_special)
        
    #     # Project importance from teacher to student
    #     student_importance = project_importance(teacher_importance, 
    #                                               teacher_tokens, 
    #                                               student_tokens, 
    #                                               token_mapping)
        
    #     # Ensure importance vectors are reshaped properly for matrix multiplication
    #     tea_mass = teacher_importance.view(-1, 1)  # Column vector
    #     stu_mass = student_importance.view(-1, 1)  # Column vector
        
    #     valid_student_seq = valid_student_seq.to(torch.float32)
    #     projected_teacher_seq = projected_teacher_seq.to(torch.float32)
        

    #     cost_matrix = pairwise_attention_distance(valid_student_seq, projected_teacher_seq)
    #     cost_matrix = cost_matrix.to(torch.float32)
    #     tea_mass = tea_mass.to(torch.float32)
    #     stu_mass = stu_mass.to(torch.float32)
    #     ot_loss, transport_plan = sinkhorn(cost_matrix, stu_mass, tea_mass)
    #     total_loss += ot_loss
    
    # avg_loss = total_loss / batch_size
    
    # return avg_loss

In [ ]:
eval_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_validation.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_validation.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_validation.csv')]

eval_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_validation.csv']

eval_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_validation.csv']
test_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_test.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_test.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_test.csv')]

test_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_test.csv']

test_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_test.csv']

# model_student.to("cuda")
# eval_classification_task(model_student, test_cls_tasks)
# eval_pair_task(model_student, test_pair_tasks)
# eval_sts_task(model_student, test_sts_tasks)

In [ ]:
from torch import Tensor
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


In [ ]:
best_f1_macro = -1.0
all_preds, all_labels = [], []
total_loss = 0.0

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_steps = len(train_loader)
total_traning_steps = num_steps * epochs
warmup_ratio = 0.1

def pick_devices():
    if torch.cuda.device_count() >= 2:
        dev_s = torch.device("cuda:0")  # student
        dev_t = torch.device("cuda:1")  # teacher
    elif torch.cuda.is_available():
        print("[WARN] Only 1 GPU available -> both on cuda:0")
        dev_s = dev_t = torch.device("cuda:0")
    else:
        print("[WARN] No GPU -> CPU")
        dev_s = dev_t = torch.device("cpu")
    return dev_s, dev_t

device_s, device_t = pick_devices()

model_student.to(device_s)
model_teacher.to(device_t)
model_teacher.eval()
for p in model_teacher.parameters():
    p.requires_grad_(False)

proj_s2t = None
proj_t2s = None

optimizer = optim.AdamW(model_student.parameters(), lr=LR)
scaler = GradScaler(enabled=torch.cuda.is_available())
scheduler = get_scheduler(
    name='cosine_with_min_lr',
    optimizer=optimizer,
    num_warmup_steps=int(total_traning_steps * warmup_ratio),
    # num_warmup_steps=1,
    num_training_steps=total_traning_steps,
    scheduler_specific_kwargs={'min_lr': 2e-6}
)
if save_dir:
    os.makedirs(save_dir, exist_ok=True)

n_items = 0
use_task_loss = True

In [ ]:
def mean_pooling(last_hidden_state: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    # sum theo chiều L
    summed = (last_hidden_state * mask).sum(dim=1)               # [B, d]
    counts = mask.sum(dim=1).clamp(min=1e-9)                     # [B, 1]
    return summed / counts
w_task     = 0.5       # bi encoder
alpha_dtw  = 0.5       # KD_DTW
beta_score = 0.0       # mse matmul
devide = False

d_s = model_student.config.hidden_size
d_t = model_teacher.config.hidden_size

proj_t2s = nn.Linear(d_t, d_s, bias=False).to(device_s)

optimizer.add_param_group({"params": proj_t2s.parameters(), "lr": LR})

for epoch in range(epochs):
    model_student.train()
    total_loss, n_items = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in pbar:
        batch_s, batch_t = {}, {}
        for k, v in batch.items():
            if not torch.is_tensor(v):
                continue
            if k.endswith("_stu") or k == "labels":
                batch_s[k] = v.to(device_s, non_blocking=True)
            if k.endswith("_tea"):
                batch_t[k] = v.to(device_t, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=torch.cuda.is_available()):
            # ========== TEACHER (bi-encoder) trên device_t ==========
            with torch.inference_mode():
                t_out1 = model_teacher(
                    input_ids=batch_t["input_ids1_tea"],
                    attention_mask=batch_t["attention_mask1_tea"],
                    output_attentions=True, return_dict=True,
                            
                )
                T_last1 = t_out1.last_hidden_state   # [B, L1, d_t] 
                T_cls1  = mean_pooling(T_last1, batch_t["attention_mask1_tea"] ) 
                T_atts  = [att.to(device_s, non_blocking=True) for att in t_out1.attentions]
                T_atts_last = t_out1.attentions[-1].to(device_s, non_blocking=True)

                T_last1 = T_last1.to(device_s, non_blocking=True)
                T_cls1  = T_cls1.to(device_s, non_blocking=True)

            # ========== STUDENT (bi-encoder) trên device_s ==========
            s_out1 = model_student(
                input_ids=batch_s["input_ids1_stu"],
                attention_mask=batch_s["attention_mask1_stu"],
                output_attentions=True, return_dict=True
            )
            s_out2 = model_student(
                input_ids=batch_s["input_ids2_stu"],
                attention_mask=batch_s["attention_mask2_stu"],
                return_dict=True
            )
            S_last1 = s_out1.last_hidden_state       # [B, L1, d_s]
            S_last2 = s_out2.last_hidden_state      # [B, L2, d_s]
            S_cls1  = S_last1[:, 0, :]               # [B, d_s]
            S_cls2  = S_last2[:, 0, :]
            S_atts  = s_out1.attentions  

            # ========== (A) HARD TASK LOSS: bi-encoder STS ==========
            
            loss_task, _ = info_nce(S_cls1, S_cls2)


            # ========== (B) DSKD: align dim bằng proj, align L bằng weight align transpose ==========
            kd_sum = 0.0

            att_loss = compute_att_loss_2(
                teacher_atts=T_atts,
                student_atts=S_atts,
                input_ids_tea=batch_t["input_ids1_tea"].to(device_s),
                input_ids_stu=batch_s["input_ids1_stu"],
                attention_mask_tea=batch_t["attention_mask1_tea"].to(device_s),
                attention_mask_stu=batch_s["attention_mask1_stu"],
                tok_teacher=tok_teacher,
                tok_student=tok_student,
                k=1,
                device=device_s,
                teacher_special=teacher_special,
                student_special=student_special,
            )
                        
            kd_loss = compute_ot_loss(
                teacher_last=T_last1,
                student_last=S_last1,
                teacher_att_last=T_atts_last,
                attention_mask_teacher=batch_t["attention_mask1_tea"].to(device_s),
                attention_mask_student=batch_s["attention_mask1_stu"],
                input_ids_tea=batch_t["input_ids1_tea"].to(device_s),
                input_ids_stu=batch_s["input_ids1_stu"],
                tok_teacher=tok_teacher,
                tok_student=tok_student,
                projector=proj_t2s,
                teacher_special=teacher_special,
                student_special=student_special,
            )

            kd_sum = att_loss*0.1 + kd_loss

            # ========== TỔNG LOSS ==========
            loss = w_task * loss_task + alpha_dtw * kd_sum
            loss = loss.float()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # ----- logging -----
        bs = next(iter(batch_s.values())).size(0) 
        total_loss += loss.item() * bs
        n_items    += bs

        mem_info = {}
        for dev_id in range(torch.cuda.device_count()):
            mem_alloc = torch.cuda.memory_allocated(dev_id) / 1024**2
            mem_reserved = torch.cuda.memory_reserved(dev_id) / 1024**2
            mem_info[f"gpu{dev_id}"] = f"{mem_alloc:.0f}/{mem_reserved:.0f}MB"

        avg_loss = total_loss / max(1, n_items)
        pbar.set_postfix({"avg_loss": f"{avg_loss:.4f}", **mem_info})

        # dọn bộ nhớ
        del S_last1, S_last2, T_last1, S_cls1, S_cls2, T_cls1, loss
        torch.cuda.empty_cache()

    eval_classification_task(model_student, test_cls_tasks)
    eval_pair_task(model_student, test_pair_tasks)
    eval_sts_task(model_student, test_sts_tasks)
